In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import cvxpy as cp
import networkx as nx
import matplotlib.pyplot as plt

from time import time
from itertools import product
from matplotlib.patches import Rectangle

In [ ]:
np.random.seed(0)
n_points = 31
box_sides = np.array([2, .5])
box_min = np.zeros((n_points, n_points, len(box_sides)))
box_max = np.zeros((n_points, n_points, len(box_sides)))
grid_points = np.arange(n_points)
for i in grid_points:
    for j in grid_points:
        center = np.array([i,j])
        np.random.shuffle(box_sides)
        sides = np.diag(box_sides).dot(np.random.rand(2))
        box_min[i,j] = center - sides
        box_max[i,j] = center + sides

In [ ]:
def plot_boxes(box_min, box_max):
    for i in grid_points:
        for j in grid_points:
#             plt.text(i, j, f'({i},{j})')
            rect = Rectangle(box_min[i,j], *(box_max[i,j] - box_min[i,j]), fc='lightcyan', ec='k')#, alpha=.5)
            plt.gca().add_patch(rect)
    reach = max(box_sides)
    plt.xlim([-reach, len(grid_points) + reach - 1])
    plt.ylim([-reach, len(grid_points) + reach - 1])

plt.figure(figsize=(9, 9))
plot_boxes(box_min, box_max)

In [ ]:
s = 's'
t = 't'
vertices = [s, t]

inters = {(i,j): [] for i in range(n_points) for j in range(n_points)}
inter_boxes = {}
centers = {}
reach = int(max(2 * box_sides))
min_reach = lambda i: max(i - reach, 0)
max_reach = lambda i: min(i + reach + 1, n_points)

start = time()
for i in grid_points:
    i_range = range(i, max_reach(i))
    for j in grid_points:
        j_range = range(min_reach(j), max_reach(j))
        for ii in i_range:
            for jj in j_range:
                if ii > i or jj > j:
                    bl = np.maximum(box_min[i,j], box_min[ii,jj])
                    tr = np.minimum(box_max[i,j], box_max[ii,jj])
                    if all(bl <= tr):
                        b1 = (i,j)
                        b2 = (ii,jj)
                        inters[b1].append(b2)
                        inters[b2].append(b1)
                        v = (b1, b2)
                        vertices.append(v)
                        inter_boxes[v] = (bl, tr)
                        centers[v] = (bl + tr) / 2
time() - start

In [ ]:
start = time()
edges = []
lengths = []

def vertex(b1, b2):
    if (b2[0] >= b1[0]) and (b2[0] > b1[0] or b2[1] > b1[1]):
        return (b1, b2)
    else:
        return (b2, b1)

for b0 in inters:
    for b1, b2 in product(inters[b0], inters[b0]):
        if b1 != b2:
            u = vertex(b0, b1)
            v = vertex(b0, b2)
            edges.append((u, v))
            duv = np.linalg.norm(centers[u] - centers[v])
            lengths.append(duv)
time() - start

In [ ]:
out_s = [(s, v) for v in vertices[2:] if v[0] == (0, 0)]
inc_t = [(v, t) for v in vertices[2:] if v[1] == (n_points - 1, n_points - 1)]
aux_edges = out_s + inc_t
edges += aux_edges
lengths += [0] * len(aux_edges)

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(vertices)
G.add_edges_from(edges)
for e, l in zip(edges, lengths):
    G.edges[e]['weight'] = l

In [ ]:
tic = time()
path = nx.shortest_path(G, source=s, target=t, weight='weight')
time() - tic

In [ ]:
import cvxpy as cp

variables = {}
constraints = []
cost = 0
x_prev = None
for v in path[1:-1]:
    variables[v] = cp.Variable(2)
    constraints.append(variables[v] >= inter_boxes[v][0])
    constraints.append(variables[v] <= inter_boxes[v][1])
    if x_prev is not None:
        cost += cp.norm(variables[v] - x_prev, 2)
    x_prev = variables[v]

prob = cp.Problem(cp.Minimize(cost), constraints)
prob.solve()
traj_opt = np.array([x.value for x in variables.values()])
prob.solver_stats.solve_time

In [ ]:
traj = np.array([centers[v] for v in path[1:-1]])
length = sum(np.linalg.norm(y - x) for x, y in zip(traj[:-1], traj[1:]))

plt.figure(figsize=(9, 9))
plot_boxes(box_min, box_max)
plt.plot(*traj.T, c='r')
plt.plot(*traj_opt.T, c='b')
plt.savefig('path_planning.pdf', bbox_inches='tight')
length, prob.value